# Web Scraping e base de dados MySql utilizando o Python

## Sobre o projeto

Nesse projeto, as bibliotecas Requests e BeautifulSoup do Python são utilizadas para coletar informações dos livros mais vendidos no site das Livrarias Cultura. Um DataFrame é então criado contendo os títulos, categorias, autores e preços dos livros.

Em seguida, uma base de dados MySql é criada com as informações coletadas. Nessa etapa, o driver MySql Connector do Python é utilizado para conectar na base de dados. 

A base criada é composta por 3 tabelas: livros_preços, autores e categorias. Os relacionamentos são definidos de tal maneira que a tabela de títulos e preços faz referência às tabelas de autores e categorias.

Apesar de bastante simples, esse projeto é um excelente exercício de duas tarefas importantes quando se trabalha com dados: a coleta de dados na web e criação de bancos sql. 

## Coleta dos dados

Importação das bibliotecas para o Web Scraping

In [1]:
# biblioteca para a captura do código html com as informações do site
import requests

# biblioteca para a coleta das informações dentro do código capturado
from bs4 import BeautifulSoup

# bibliotecas padrão do python para manipulação dos dados
import pandas as pd
import numpy as np

Realizando uma requisição para obter os dados da página 

In [2]:
# endereço da web
url = 'https://www3.livrariacultura.com.br/hotsites/maisvendidos'

# obtendo as informações da página
response = requests.get(url)

Utilizando o BeautifulSoup para coletar os dados do código html

In [3]:
soup = BeautifulSoup(response.content, 'html.parser')

Coletando os títulos dos livros e as informações dos autores

In [4]:
livros = []
autores = []

for item in soup.find_all('div', attrs = {'class' : 'prateleiraProduto__informacao'}):
    livros.append(item.find('h2').text.strip())
    autores.append(item.find('h3').find('li').text)

Coletando as categorias

In [5]:
categorias = []

for item in soup.find_all('div', attrs = {'class' : 'prateleiraProduto__informacao__categoria'}): 
    categorias.append(item.text.strip())

Coletando os preços

In [6]:
precos = []

for item in soup.find_all('div', attrs = {'class' : 'prateleiraProduto__informacao__preco'}):
    precos.append((item.find('span').text.strip('BRL $').replace(",", ".")))

Criando um dataframe com os dados coletados.

In [7]:
bestsellers = pd.DataFrame({'livros' : livros,
                            'autores': autores,
                           'categorias' : categorias,
                          'precos' : precos})

In [8]:
bestsellers

,livros,autores,categorias,precos
0,FAZENDA DOS ANIMAIS (CAPA DURA),"Autor:ORWELL, GEORGE",Literatura Internacional,79.90
1,A TEMPERATURA ENTRE VOCÊ E EU,"Autor:ZEPKA, BRIAN",Ficção Fantasiosa,54.90
2,FLORES PARA ALGERNON,"Autor:KEYES, DANIEL|Tradutor:GEISLER, LUISA",Ficção Cientifica,69.90
3,AMOR & GELATO,"Tradutor:FARO, JOANA|Autor:WELCH, JENNA EVANS",Literatura,54.90
4,É ASSIM QUE ACABA,"Autor:HOOVER, COLLEEN|Tradutor:CATAO, PRISCILA",Romances,49.90
...,...,...,...,...
87,O PODER DO HÁBITO,"Tradutor:MANTOVANI, RAFAEL|Autor:DUHIGG, CHARLES",Desenvolvimento Profissional,82.90
88,ESPECIALISTA EM PESSOAS,"Autor:BRUNET, TIAGO",Autoajuda,59.90
89,COMO FAZER AMIGOS E INFLUENCIAR PESSOAS,"Autor:CARNEGIE, DALE|Tradutor:ALMEIDA, LIVIA DE",Desenvolvimento Pessoal,54.90
90,A CORAGEM DE SER IMPERFEITO,"Autor:BROWN, BRENE|Tradutor:MACEDO, JOEL",Desenvolvimento Pessoal,44.90


## Migração dos dados para uma base MySql

### Conectando na base

In [9]:
# driver do python para conexão com a base mysql
import mysql.connector

In [10]:
# conectando na base "bestsellers_sql"
connection = mysql.connector.connect(
            host="localhost",
            user="root",
            passwd="minhasenhadosql",
            database="bestsellers_sql"  
        )

In [11]:
cursor = connection.cursor()

### Criando as tabelas

Os dados extraídos serão armazenados em 3 tabelas: livros_precos, autores e categorias. A tabela livros-precos será a tabela principal, fazendo referências para as tabelas "autores" e "categorias" através de chaves secundárias.

In [12]:
# criando a tabela "autores"
cursor.execute("""CREATE TABLE autores
(
	autores_id int unsigned not null auto_increment,
    nome_autor varchar(100) not null,
    PRIMARY KEY (autores_id)
);"""
)

In [13]:
# criando a tabela "categorias"
cursor.execute("""CREATE TABLE categorias
(
	categorias_id int unsigned not null auto_increment,
    categoria varchar(50) not null,
    PRIMARY KEY (categorias_id)
);"""
)

In [16]:
# criando a tabela "livros-precos"
cursor.execute("""CREATE TABLE livros_precos
(
	livros_id int unsigned not null auto_increment,
    titulo varchar(100) not null,
    autor_id int unsigned not null,
    categoria_id int unsigned not null,
    preco double,
    PRIMARY KEY (livros_id),
    FOREIGN KEY (autor_id) REFERENCES autores(autores_id),
    FOREIGN KEY (categoria_id) REFERENCES categorias(categorias_id)        
);"""
)

### Inserindo os dados nas tabelas

Inicialmente a tabela de autores é preenchida. Como existem autores com mais de um livro na lista, o comando "unique" é utilizado para que não existam registros duplicados.

In [19]:
# garante que os índices da tabela "autores" se iniciem no valor 1 
cursor.execute("""ALTER TABLE autores AUTO_INCREMENT=1;""")

# comando sql para a inserção de valores na tabela "autores"
sql = "INSERT INTO autores (nome_autor) VALUES (%s);"

# gerando uma tupla a partir da coluna "autores" do pandas dataframe
val = tuple(zip(bestsellers['autores'].unique()))

# executando o comando de inserção de múltiplos valores
cursor.executemany(sql, val)

# confirmando a execução da inserção
connection.commit()

A seguir, a tabela "categorias" é preenchida. Novamente o comando "unique" é utilizado para eliminar categorias repetidas.

In [18]:
# garante que os índices da tabela "categoria" se iniciem no valor 1 
cursor.execute("""ALTER TABLE categorias AUTO_INCREMENT=1;""")

# comando sql para a inserção de valores na tabela "categorias"
sql = "INSERT INTO categorias (categoria) VALUES (%s);"

# gerando uma tupla a partir da coluna "autores" do pandas dataframe
val = tuple(zip(bestsellers['categorias'].unique()))

# executando o comando de inserção de múltiplos valores
cursor.executemany(sql, val)

# confirmando a execução da inserção
connection.commit()

Para o preenchimento da tabela "livros_precos" será necessário identificar os id's correspondentes dos autores e categorias. 

Abaixo, os nomes únicos dos autores do dataframe são capturados e, em seguida, os id's são identificados.

In [21]:
# nomes únicos dos autores
autores = bestsellers['autores'].unique() 

In [22]:
# identificando os id's dos autores

# lista para guardar os valores de id
autor_id = []

# percorre os nomes dos autores no dataframe e adiciona o valor do id na lista "autor_id"
for nome in bestsellers['autores']:
    idx = np.where(autores == nome)[0][0]
    autor_id.append(int(idx+1))

Um procedimento semelhante é agora realizado para a identificação dos id's das categorias.

In [24]:
# nomes únicos das categorias
categorias = bestsellers['categorias'].unique() 

In [25]:
# identificando os id's das categorias

# lista para guardar os valores de id
categoria_id = []

# percorre os nomes das categorias no dataframe e adiciona o valor do id nas lista "categoria_id"
for classe in bestsellers['categorias']:
    idx = np.where(categorias == classe)[0][0]
    categoria_id.append(int(idx+1))

Por fim, os valores da tabela "livros_precos" são agora preenchidos.

In [26]:
# comando sql para a inserção de valores na tabela "livros_precos"
sql = "INSERT INTO livros_precos (titulo, autor_id, categoria_id, preco) VALUES (%s, %s, %s, %s);"

# gerando uma tupla com os dados para serem inseridos na tabela
val = tuple(zip(bestsellers['livros'], autor_id, categoria_id, bestsellers['precos']))

# executando o comando de inserção de múltiplos valores
cursor.executemany(sql, val)

# confirmando a execução da inserção
connection.commit()